In [1]:
import numpy as np  
import umap.umap_ as umap
from sklearn.model_selection import train_test_split  
from sklearn.datasets import fetch_openml  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense  
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
# Load dataset  
mnist = fetch_openml('mnist_784', version=1)  
X, y = mnist.data, mnist.target  

# encode  
y = np.array([label for label in y])  

from tensorflow.keras.utils import to_categorical

y = to_categorical(y) 

In [3]:
# Using UMAP to reduce dimensions 
umap_model = umap.UMAP(n_components=50)
X_umap = umap_model.fit_transform(X)  

In [4]:
# Split dataset  
X_train, X_test, y_train, y_test = train_test_split(X_umap, y, test_size=0.2, random_state=42)  

# Initialize model  
model = Sequential()  
model.add(Dense(128, input_dim=50, activation='relu'))  # define input_num=50
model.add(Dense(10, activation='softmax'))  # define output classfication  

d:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# complie model 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [6]:
# train model  
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))  

Epoch 1/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7916 - loss: 0.8915 - val_accuracy: 0.9645 - val_loss: 0.2249
Epoch 2/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9630 - loss: 0.2155 - val_accuracy: 0.9689 - val_loss: 0.1937
Epoch 3/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9665 - loss: 0.1986 - val_accuracy: 0.9694 - val_loss: 0.1974
Epoch 4/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9671 - loss: 0.1945 - val_accuracy: 0.9701 - val_loss: 0.2019
Epoch 5/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9672 - loss: 0.1897 - val_accuracy: 0.9658 - val_loss: 0.2140
Epoch 6/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9666 - loss: 0.1902 - val_accuracy: 0.9617 - val_loss: 0.2436
Epoch 7/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9688 - loss: 0.1819 - val_accuracy: 0.9690 - val_loss: 0.1871
Epoch 8/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9666 - loss: 0.1920 - 

In [17]:
import tensorflow.saved_model  # save this model

In [21]:
tensorflow.saved_model.save(model,'./UMAP_pretrained')

INFO:tensorflow:Assets written to: ./UMAP_pretrained\assets


INFO:tensorflow:Assets written to: ./UMAP_pretrained\assets


In [9]:
from PIL import Image
import numpy as np
import os

In [27]:
# Define a image preprocess function 
def preprocess_image(image_path):  
    image = Image.open(image_path).convert('L')    
    image = image.resize((28, 28), Image.BILINEAR )  
    image_array = np.array(image)  
    return image_array  
  
#Define a dimension reduction function 
def reduce_dimensionality(image_array, umap_model):  
    return umap_model.transform(image_array.reshape(1, -1))  
  
#Define a prediction function  
def predict_image(image_path, umap_model, trained_model):  
      
    image_array = preprocess_image(image_path)  

    reduced_image = reduce_dimensionality(image_array, umap_model)  
        
    prediction = trained_model.predict(reduced_image)  
    predicted_class = np.argmax(prediction)  
      
    return predicted_class, prediction  

In [33]:
# Load a image randomly
image_path = 'C:/Users/Administrator/Desktop/7&14_JUN_Deep learning/Individual Proposal/hw/img_5.jpg'
 
Image.open(image_path)

In [34]:
# Predict image
predicted_class, prediction = predict_image(image_path, umap_model, model)  
print(f"Predicted digital: {predicted_class}")  
print(f"Prediction probabilities: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted digital: 3
Prediction probabilities: [[6.5268563e-05 1.5278711e-04 4.4366391e-03 9.8901165e-01 3.1571403e-06
  2.7982860e-03 6.8009875e-05 9.2136666e-05 2.5106922e-03 8.6148229e-04]]
